<a href="https://colab.research.google.com/github/Komal0103/CUDA-matrix-exponential/blob/main/matrix-exponential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time
# from scipy.linalg._misc import norm
import cupy as cp
from cupy.linalg import norm
import math
# import ipdb

import numba
from numba import cuda
import pdb
from pdb import set_trace

In [ ]:
@cuda.jit(device=True, inline=True)
def matmul(a, b, c, m, n, k):
  # implement tiling here
  row = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
  col = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
  if (row<m and col<k):
    sum=0
    for i in range(n):
      # sum=0
      A=a[row*n+i]
      B=b[col+i*k]
      sum=sum+A*B
    c[row*k+col]=sum

@cuda.jit(device=True, inline=True)
def addMatrices(a, b, c, n, threadId):
  row = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
  col = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
  if (row<n and col<n):
    for i in range(n):
      c[row*n+col]=a[row*n+col]+b[row*n+col]
  # if threadId<(n**2):
    # c[threadId]=a[threadId]+b[threadId]

@cuda.jit
def squaring(E, eA, S, n):
  s=S[0]
  for k in range(s):
    # a possible problem here
    matmul(E, E, eA, n, n, n)
    if s>1:
      E=eA

@cuda.jit
def computeA3(a, a2, a3, n):
  matmul(a, a2, a3, n, n, n)

@cuda.jit
def computeA4(a, a2, s, temp, i, E, n):
  blockId=cuda.blockIdx.x+cuda.blockIdx.y*cuda.gridDim.x
  threadId=blockId * (cuda.blockDim.x * cuda.blockDim.y) + (cuda.threadIdx.y * cuda.blockDim.x) + cuda.threadIdx.x
  matmul(a2, s, temp, n, n, n)
  if threadId<n*n:
    E[threadId]=i[threadId]+a[threadId]+temp[threadId]

@cuda.jit
def computeA8(a, a2, a4, a8, i, temp, temp2, E, n):
  x3=[2/3]
  coeff=[1/88*(1+cp.sqrt(177))*x3, 1/352*(1+cp.sqrt(177))*x3, 1, 1, 1/630*(857-58*cp.sqrt(177)),
         (-271+29*cp.sqrt(177))/(315*x3), (11*(-1+cp.sqrt(177)))/(1260*x3), (11*(-9+cp.sqrt(177)))/(5040*x3),
         -((-89+cp.sqrt(177))/(5040*x3^2))]
  # error here-thread idexing issue
  # get the absolute grid index of the thread, and then make sure it is less than n*n (total number of elements in the array)
  blockId=cuda.blockIdx.x+cuda.blockIdx.y*cuda.gridDim.x
  threadId=blockId * (cuda.blockDim.x * cuda.blockDim.y) + (cuda.threadIdx.y * cuda.blockDim.x) + cuda.threadIdx.x
  if threadId<n*n:
    temp[threadId]=coeff[0]*a[threadId]+coeff[1]*a2[threadId]
  matmul(a2, temp, a4, n, n, n)
  if threadId<n*n:
    temp[threadId]=x3[0]*a2[threadId]+a4
    temp2[threadId]=coeff[5]*i[threadId]+coeff[6]*a+coeff[7]*a2[threadId]+coeff[8]*a4
  matmul(temp, temp2, a8, n, n, n)
  if threadId<n*n:
    # TODO: check for missing term
    E[threadId]=i[threadId]+a[threadId]+coeff[4]*a2[threadId]+a8[threadId]

@cuda.jit
def computeA18(a, a2, a3, a6, i, temp1, temp2, temp3, temp4, temp5, temp6, temp7, temp8, temp9, E, coeff, n):
  # TODO: consider making an array of a class for all the temporary variables here, inside the kernel, if possible
  blockId=cuda.blockIdx.x+cuda.blockIdx.y*cuda.gridDim.x
  threadId=blockId * (cuda.blockDim.x * cuda.blockDim.y) + (cuda.threadIdx.y * cuda.blockDim.x) + cuda.threadIdx.x
  if threadId<n**2:
    a6[threadId]=0
    temp1[threadId]=0
    temp2[threadId]=0
    temp3[threadId]=0
    temp4[threadId]=0
    temp5[threadId]=0
    temp6[threadId]=0
    temp7[threadId]=0
    temp8[threadId]=0
    temp9[threadId]=0
  matmul(a3, a3, a6, n, n, n)
  if threadId<(n**2):
    temp1[threadId]=coeff[1]*a[threadId]+coeff[2]*a2[threadId]+coeff[3]*a3[threadId]
    temp2[threadId]=coeff[5]*a[threadId]+coeff[6]*a2[threadId]+coeff[7]*a3[threadId]+coeff[8]*a6[threadId]
    temp3[threadId]=coeff[9]*i[threadId]+coeff[10]*a[threadId]+coeff[11]*a2[threadId]+coeff[12]*a3[threadId]+coeff[13]*a6[threadId]
    temp4[threadId]=coeff[14]*i[threadId]+coeff[15]*a[threadId]+coeff[16]*a2[threadId]+coeff[17]*a3[threadId]+coeff[18]*a6[threadId]
    temp5[threadId]=coeff[21]*a2[threadId]+coeff[22]*a3[threadId]+coeff[23]*a6[threadId]
  matmul(temp1, temp5, temp6, n, n, n)
  cuda.syncthreads()
  addMatrices(temp4, temp6, temp7, n, threadId)
  # cuda.syncthreads()
  # if threadId<(n**2):
    # temp7[threadId]=temp4[threadId]+temp6[threadId]
  if threadId<(n**2):
    # try to reuse temp6 here
    temp9[threadId]=temp3[threadId]+temp7[threadId]
  matmul(temp9, temp7, temp8, n, n, n)
  cuda.syncthreads()
  if threadId<(n**2):
    E[threadId]=temp2[threadId]+temp8[threadId]

@cuda.jit
def computeA12(a, a2, a3, i, temp1, temp2, temp3, temp4, temp5, temp6, temp7, E, coeff, n):
  blockId=cuda.blockIdx.x+cuda.blockIdx.y*cuda.gridDim.x
  threadId=blockId * (cuda.blockDim.x * cuda.blockDim.y) + (cuda.threadIdx.y * cuda.blockDim.x) + cuda.threadIdx.x
  # TODO: check once
  if threadId<n*n:
    temp1=coeff[0]*i+coeff[4]*a+coeff[8]*a2+coeff[12]*a3
    temp2=coeff[1]*i+coeff[5]*a+coeff[9]*a2+coeff[13]*a3
    temp3=coeff[2]*i+coeff[6]*a+coeff[10]*a2+coeff[14]*a3
    temp4=coeff[3]*i+coeff[7]*a+coeff[11]*a2+coeff[15]*a3
    matmul(temp4, temp4, temp5, n, n, n)
    if threadId<n*n:
      temp6[threadId]=temp3[threadId]+temp5[threadId]
    if threadId<n*n:
      # attempting to reuse temp5
      temp5[threadId]=temp2[threadId]+temp6[threadId]
    matmul(temp5, temp6, temp7, n, n, n)
    if threadId<n*n:
      E[threadId]=temp1[threadId]+temp7[threadId]

In [ ]:
def TaylorAlgorithm(a, degree, n, TILE_SIZE):
  # blocks_per_grid=(math.ceil((n-1)/TILE_SIZE)+1, math.ceil((n-1)/TILE_SIZE)+1)
  blocks_per_grid=(2, 2)
  threads_per_block=(TILE_SIZE, TILE_SIZE)
  dev_a=cuda.to_device(a)
  if degree>=2:
    dev_a2=cuda.device_array([n**2,], dtype="complex")
    s=[1]
    dev_s=cuda.to_device(s)
    # CUDA inlines functions automatically-matmul is a device function
    squaring[blocks_per_grid, threads_per_block](dev_a, dev_a2, dev_s, n)
    cuda.synchronize()
    a2=dev_a2.copy_to_host()
  if degree>=8:
    dev_a3=cuda.device_array([n**2], dtype="complex")
    # matmul is a device function and cannot be called by the host
    computeA3[blocks_per_grid, threads_per_block](dev_a, dev_a2, dev_a3, n)
    a3=dev_a3.copy_to_host()
    cuda.synchronize()
  if degree==1:
    E=identity(n)+a
  elif degree==2:
    E=identity(n)+a+a2/2
  elif degree==4:
    i=identity(n)
    dev_i=cuda.to_device(i)
    s=identity(n)/2+a/6+a2/24
    dev_s=cuda.to_device(s)
    dev_temp=cuda.device_array([n*n,], dtype="complex")
    dev_E=cuda.device_array([n*n,], dtype="complex")
    computeA4[blocks_per_grid, threads_per_block](dev_a, dev_a2, dev_s, dev_temp, dev_i, dev_E, n)
    cuda.synchronize()
    E=dev_E.copy_to_host()
  elif degree==8:
    i=identity(n)
    I=i.reshape([n*n,])
    dev_i=cuda.to_device(I)
    dev_a4=cuda.device_array([n*n,], dtype="complex")
    dev_a8=cuda.device_array([n*n,], dtype="complex")
    temp=cuda.device_array([n*n,], dtype="complex")
    temp2=cuda.device_array([n*n,], dtype="complex")
    dev_E=cuda.device_array([n*n,], dtype="complex")
    computeA8[blocks_per_grid, threads_per_block](dev_a, dev_a2, dev_a4, dev_a8, dev_i, temp, temp2, dev_E, n)
    cuda.synchronize()
    E=dev_E.copy_to_host()
  elif degree==12:
    coeff=np.asarray([-0.0186023205146205532243437300433, 4.6, 0.211693118299809442949323323336, 0,-0.00500702322573317730979741843919, 0.992875103538486836140479571505,
         0.158224384715726725371768893252, -0.131810610138301840156819349464, -0.573420122960522263905952420789,
         -0.132445561052799638845074997454, 0.165635169436727415011171668419, -0.0202785554058925907933568229945,
         -0.133399693943892059700768926983, 0.0017299, 0.0107862779315792425026320640108, -0.00675951846863086359778560766482])
    dev_coeff=cuda.to_device(coeff)
    i=identity(n)
    I=i.reshape([n*n,])
    dev_i=cuda.to_device(I)
    dev_a3=cuda.device_array([n*n,], dtype="complex")
    dev_a8=cuda.device_array([n*n,], dtype="complex")
    temp1=cuda.device_array([n*n,], dtype="complex")
    temp2=cuda.device_array([n*n,], dtype="complex")
    temp3=cuda.device_array([n*n,], dtype="complex")
    temp4=cuda.device_array([n*n,], dtype="complex")
    temp5=cuda.device_array([n*n,], dtype="complex")
    temp6=cuda.device_array([n*n,], dtype="complex")
    temp7=cuda.device_array([n*n,], dtype="complex")
    dev_E=cuda.device_array([n*n,], dtype="complex")
    computeA12[blocks_per_grid, threads_per_block](dev_a, dev_a2, dev_a3, dev_i, temp1, temp2, temp3, temp4, temp5, temp6, temp7, dev_E, dev_coeff, n)
    cuda.synchronize()
    E=dev_E.copy_to_host()
  elif degree==18:
    # possible problem here-in computeA18
    coeff=cp.asarray([0, -0.100365581030144620014614939405, -0.00802924648241156960116919515244, -0.000892138498045729955685466128049, 0,
         0.397849749499645076145196, 1.367837784604117199225237068782228242, 0.49828962252538267755685881726227335, -0.0006378981945947233092415500564919,
         -10.9676396052962062593517462675368412, 1.68015813878906197182785401724810509, 0.05717798464788655127028717132252481317,
         -0.00698210122488052084290466483801553, 0.00003349750170860705383133673406684398, -0.0904316832390810561971468877018349549,
         -0.0676404519071381907560079900379707485, 0.0675961301770459646082799195078937, 0.02955525704293155274260691822422312437,
         -0.00001391802575160607011247399793437, 0, 0, -0.0923364619367118592764570775143, -0.0169364939002081717191385115723,
         -0.0000140086798182036159794363205726])
    dev_coeff=cuda.to_device(coeff)
    i=identity(n)
    I=i.reshape([n**2])
    dev_i=cuda.to_device(I)
    dev_a6=cuda.device_array([n**2], dtype="complex")
    temp1=cuda.device_array([n**2], dtype="complex")
    temp2=cuda.device_array([n**2], dtype="complex")
    temp3=cuda.device_array([n**2], dtype="complex")
    temp4=cuda.device_array([n**2], dtype="complex")
    temp5=cuda.device_array([n**2], dtype="complex")
    temp6=cuda.device_array([n**2], dtype="complex")
    temp7=cuda.device_array([n**2], dtype="complex")
    temp8=cuda.device_array([n**2], dtype="complex")
    temp9=cuda.device_array([n**2], dtype="complex")
    dev_E=cuda.device_array([n**2], dtype="complex")
    computeA18[blocks_per_grid, threads_per_block](dev_a, dev_a2, dev_a3, dev_a6, dev_i, temp1, temp2, temp3, temp4, temp5, temp6, temp7, temp8, temp9, dev_E, dev_coeff, n)
    cuda.synchronize()
    E=dev_E.copy_to_host()
    # host_temp4=temp4.copy_to_host()
    # print(host_temp4)
    # print("------------------------------------------------------------------")
    # host_temp6=temp6.copy_to_host()
    # print(host_temp6)
    # print("------------------------------------------------------------------")
    # host_temp7=temp7.copy_to_host()
    # print(host_temp7)
    # print("------------------------------------------------------------------")
    # host_temp3=temp3.copy_to_host()
    # print(host_temp3)
    # print("------------------------------------------------------------------")
    # host_temp9=temp9.copy_to_host()
    # print(host_temp9)
    # print("------------------------------------------------------------------")

    return E

In [ ]:
def identity(n):
    ident=np.zeros([n,n], dtype="complex")
    for row in range(n):
        for column in range(n):
            if (row==column):
                ident[row][column]=1
    return ident

def exponential(a, n, TILE_SIZE):
    start=time.perf_counter()
    # converting to 1D array for matrix multiplication
    A=a.reshape([n*n,])
    anorm=cp.linalg.norm(a, 1)
    theta=[2.220446049250313e-16, 2.580956802971767e-08, 1.386347866119121e-05, 3.397168839976962e-04, 2.400876357887274e-03, 9.065656407595102e-03, 2.384455532500274e-02,
         4.991228871115323e-02, 8.957760203223343e-02, 1.441829761614378e-01, 2.142358068451711e-01, 2.996158913811580e-01, 3.997775336316795e-01, 5.139146936124294e-01, 6.410835233041199e-01,
         7.802874256626574e-01, 9.305328460786568e-01, 1.090863719290036]
    if anorm <= theta[17]:
      for k in (1, 2, 4, 8, 12):
        if anorm <= theta[k]:
          # return Taylor Polynomial of the order k
          eA=TaylorAlgorithm(A, k, n, TILE_SIZE)

    if anorm>theta[17]:
      # find characteristic and mantissa of log2
      value=anorm/theta[17]
      s=0
      while (s<=int(value)):
          if (2**s>int(value)):
              break
          s+=1
      t=2**(math.log2(value)-s)
      if t==0.50000:
          s=s-1
      As=A/(2**s)
      S=[s]
      dev_S=cuda.to_device(S)
      E=TaylorAlgorithm(As, 18, n, TILE_SIZE)
      dev_E=cuda.to_device(E)
      dev_eA=cuda.device_array([n*n,], dtype="complex")
      # blocks_per_grid=(math.ceil((n-1)/TILE_SIZE)+1, math.ceil((n-1)/TILE_SIZE)+1)
      blocks_per_grid=(2, 2)
      threads_per_block=(TILE_SIZE, TILE_SIZE)
      squaring[blocks_per_grid, threads_per_block](dev_E, dev_eA, dev_S, n)
      cuda.synchronize()
      eA=dev_eA.copy_to_host()
      eA=eA.reshape([n,n])

    end=time.perf_counter()
    return eA, start, end

In [ ]:
# possible problems
# overscaling-high
# inaccurate tiling size
# implement tiling algorithm
# driver function
TILE_SIZE=30
n=75
# declaring a as a 2D array and converting to a 1D array when required (for matrix multiplication)
a=np.zeros([n, n], dtype="complex")
for k in range(n):
    for l in range(n):
        a[k][l]=1+1j
# print(a)
eA, start, end=exponential(a, n, TILE_SIZE)
print(eA)
print("Execution Time %0.7f" %(end-start))

[[-7.31489173e+41+1.58911840e+42j -6.63286771e+41+1.59094966e+42j
   7.13315786e+37+1.19257498e+38j ...  7.81250000e-03+7.81250000e-03j
   7.81250000e-03+7.81250000e-03j  7.81250000e-03+7.81250000e-03j]
 [ 4.48353393e+49-3.25885298e+46j  4.41458506e+49-1.63952996e+48j
  -5.43846657e+11-3.25210574e+13j ...  7.81250000e-03+7.81250000e-03j
   7.81250000e-03+7.81250000e-03j  7.81250000e-03+7.81250000e-03j]
 [ 3.61312021e+13+2.24187187e+13j  3.55137969e+13+2.19991058e+13j
   1.13724757e+13+2.31346579e+13j ...  7.81250000e-03+7.81250000e-03j
   7.81250000e-03+7.81250000e-03j  7.81250000e-03+7.81250000e-03j]
 ...
 [ 7.81250000e-03+7.81250000e-03j  7.81250000e-03+7.81250000e-03j
   7.81250000e-03+7.81250000e-03j ...  7.81250000e-03+7.81250000e-03j
   7.81250000e-03+7.81250000e-03j  7.81250000e-03+7.81250000e-03j]
 [ 7.81250000e-03+7.81250000e-03j  7.81250000e-03+7.81250000e-03j
   7.81250000e-03+7.81250000e-03j ...  7.81250000e-03+7.81250000e-03j
   7.81250000e-03+7.81250000e-03j  7.81250000e-

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
